# Pig.E Bank Data Cleaning 

## Table of Contents

### 1. Import libraries and data.
### 2. Check data shape, structure, and basic descriptive statistics

### 3. Consistency Checks

#### 3.1 Missing values
#### 3.2 Mixed-type data
#### 3.3 Update data types
#### 3.4 Duplicates

### 4. Data Wrangling
#### 4.1 Drop columns if necessary

### 5. Check frequencies in pertinent columns
#### 5.1 Country
#### 5.2 Gender

### 6. Check shape, structure and basic descriptive statistics after updates

### 7. Export data

## Importing Libraries and Data

In [99]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [100]:
# Set path
path = r'/Users/amritkaur/Desktop/CF Data Immersion/Chapter 5/Pig E Bank Money Laundering Analysis'

# Import data
bank = pd.read_csv(os.path.join(path, '02. Data', 'Original Data', 'Raw Data.csv'), index_col = False)

## 2. Check data shape, structure, and basic descriptive statistics

In [102]:
bank.head()

,Row_Number,Customer_ID,Last_Name,Credit Score,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard?,IsActiveMember,Estimated Salary,ExitedFromBank?,Unnamed: 14,Unnamed: 15
0,1,15634602,Hargrave,619.0,France,Female,42.0,2,$0.00,1,1,1,"$101,348.88",1,NaN,NaN
1,2,15647311,Hill,608.0,Spain,Female,41.0,1,"$83,807.86",1,0,1,"$112,542.58",0,NaN,NaN
2,3,15619304,Onio,502.0,France,Female,42.0,8,"$159,660.80",3,1,0,"$113,931.57",1,NaN,NaN
3,4,15701354,Boni,699.0,France,Female,39.0,1,$0.00,2,0,0,"$93,826.63",0,NaN,NaN
4,5,15737888,Mitchell,850.0,Spain,Female,43.0,2,"$125,510.82",1,1,1,"$79,084.10",0,NaN,NaN


In [103]:
bank.describe()

,Row_Number,Customer_ID,Credit Score,Age,Tenure,NumOfProducts,HasCrCard?,IsActiveMember,ExitedFromBank?,Unnamed: 14,Unnamed: 15
count,991.00000,9.910000e+02,988.000000,990.000000,991.000000,991.000000,991.000000,991.000000,991.000000,0.0,0.0
mean,496.00000,1.569232e+07,648.512146,38.684848,5.066599,1.522704,0.706357,0.507568,0.205853,NaN,NaN
std,286.22136,7.223333e+04,98.202853,10.958220,2.928371,0.573170,0.455660,0.500195,0.404527,NaN,NaN
min,1.00000,1.556609e+07,376.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,NaN,NaN
25%,248.50000,1.563019e+07,579.750000,32.000000,2.000000,1.000000,0.000000,0.000000,0.000000,NaN,NaN
50%,496.00000,1.569275e+07,653.500000,37.000000,5.000000,1.000000,1.000000,1.000000,0.000000,NaN,NaN
75%,743.50000,1.575379e+07,721.000000,44.000000,8.000000,2.000000,1.000000,1.000000,0.000000,NaN,NaN
max,991.00000,1.581536e+07,850.000000,82.000000,10.000000,4.000000,1.000000,1.000000,1.000000,NaN,NaN


In [104]:
bank.shape

(991, 16)

Initial Data Insights:
There are 991 rows and looks like we have 3 missing values in credit scores because there are 988 rows.
990 age: possibly missing 1 value here
989 estimated salaries: possibly missing 2 values here
min and max credit scores seems fine.
min age: 2 (unrealistic unless someone opened an account for their child. This was confirmed with stakeholders that these are likely outliers and can be removed from the dataset.)

## 3. Consistency Checks

### 3.1 Missing values

In [108]:
# Check for missing values
bank.isnull().sum()

Row_Number            0
Customer_ID           0
Last_Name             1
Credit Score          3
Country               0
Gender                1
Age                   1
Tenure                0
Balance               0
NumOfProducts         0
HasCrCard?            0
IsActiveMember        0
Estimated Salary      2
ExitedFromBank?       0
Unnamed: 14         991
Unnamed: 15         991
dtype: int64

#### 1. Country – Some are abbreviated, some are spelt out. Changed the abbreviations from DE to Germany, ES to Spain and FR to France. Fix the spelling of France  for clarity and consistency
#### 2. Age – There are rows with age 2. I  got the average age of females  and impute it on the table. The average age value is 39.
#### 3. Last_name – there are some names with question marks. Due to PII, I think it’s not important for analysis so I just remove the ? Sign from row 11. 
#### 4. Gender – Some are abbreviated, some are spelt out. There is one NULL value. I changed  the abbreviated ones and leave the NULL value as is.  Some clients are not comfortable to disclose their gender due to PII. 
#### 5. Credit Score – There are 3 blank values. I imputed the Blank values with Average Credit score valu because this column is important for our analysis. 
#### 6. Estimated salary – There is a blank and NULL value cells. As per Germany' male average salary and Spain's Female Average salary, I impute the values as it might be important factor for furute analysis. ysis. 

### Last_Name

In [111]:
# Fill in missing 'Last_Name' value with 'Unknown'
bank['Last_Name'] = bank['Last_Name'].fillna('Unknown')

In [112]:
# Check for missing values in last_name
bank['Last_Name'].isnull().sum()

0

### Credit Score

In [114]:
# Calculate the mean credit score
mean_credit_score = bank['Credit Score'].mean()

# Fill missing values in the credit score column with the mean_credit_score
bank.loc[:, 'Credit Score'] = bank['Credit Score'].fillna(mean_credit_score)

In [115]:
# Check for missing values in Credit Score
bank['Credit Score'].isnull().sum()

0

### Gender

In [117]:
# Drop record with missing gender value
bank = bank.dropna(subset=['Gender'])

In [118]:
# Check for missing values in Gender
bank['Gender'].isnull().sum()

0

### Age

In [120]:
# Calculate the mean age
mean_age = bank['Age'].mean()

# Fill missing value in Age
bank.loc[:, 'Age'] = bank['Age'].fillna(mean_age)

In [121]:
# Check for missing values in Age
bank['Age'].isnull().sum()

0

### Estimated Salary

In [123]:
## In order to perform aggregation on 'Estimated Salary', we must remove the dollar sign and comma and convert it from a string, into a numberical data type

# Remove dollar signs and commas on 'Estimated Salary' column and change data type to float64
bank['Estimated Salary'] = bank['Estimated Salary'].replace(r'[\$,]', '', regex=True).astype('float64')

# Calculate the mean estimated salary
mean_estimated_salary = bank['Estimated Salary'].mean()

# Fill missing values in estimated salary
bank.loc[:, 'Estimated Salary'] = bank['Estimated Salary'].fillna(mean_estimated_salary)

In [124]:
# Check for missing values in Estimated Salary
bank['Estimated Salary'].isnull().sum()

0

In [125]:
# Final check
bank.isnull().sum()

Row_Number            0
Customer_ID           0
Last_Name             0
Credit Score          0
Country               0
Gender                0
Age                   0
Tenure                0
Balance               0
NumOfProducts         0
HasCrCard?            0
IsActiveMember        0
Estimated Salary      0
ExitedFromBank?       0
Unnamed: 14         990
Unnamed: 15         990
dtype: int64

There are no more missing values left.

### 3.2 Mixed-type data

In [128]:
# Check for mixed type data

for col in bank.columns.tolist():
    weird = (bank[[col]].map(type) != bank[[col]].iloc[0].apply(type)).any(axis = 1)
    if len(bank[weird]) > 0:
        print (col)
    else: 
        print('No Mixed-type data')

No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data
No Mixed-type data


### 3.3 Update data types

In [130]:
# Check data types
bank.dtypes

Row_Number            int64
Customer_ID           int64
Last_Name            object
Credit Score        float64
Country              object
Gender               object
Age                 float64
Tenure                int64
Balance              object
NumOfProducts         int64
HasCrCard?            int64
IsActiveMember        int64
Estimated Salary    float64
ExitedFromBank?       int64
Unnamed: 14         float64
Unnamed: 15         float64
dtype: object

In [131]:
# Change data type on 'Credit Score' to int64
bank['Credit Score'] = bank['Credit Score'].astype('int64')

In [132]:
# Change data type on 'Age' to int64
bank['Age'] = bank['Age'].astype('int64')

In [155]:
# Remove dollar signs and commas on 'Balance' column and change data type to float64
bank['Balance'] = bank['Balance'].replace(r'[\$,]', '', regex=True).astype('float64')

In [157]:
# Change data type on 'HasCrCard?' to boolean
bank['HasCrCard?'] = bank['HasCrCard?'].astype('boolean')

In [159]:
# Change data type on 'IsActiveMember' to boolean
bank['IsActiveMember'] = bank['IsActiveMember'].astype('boolean')

In [161]:
# Change data type on 'ExitedFromBank?' to boolean
bank['ExitedFromBank?'] = bank['ExitedFromBank?'].astype('boolean')

In [163]:
# Confirm datatypes changes
bank.dtypes

Row_Number            int64
Customer_ID           int64
Last_Name            object
Credit Score          int64
Country              object
Gender               object
Age                   int64
Tenure                int64
Balance             float64
NumOfProducts         int64
HasCrCard?          boolean
IsActiveMember      boolean
Estimated Salary    float64
ExitedFromBank?     boolean
Unnamed: 14         float64
Unnamed: 15         float64
dtype: object

### 3.4 Duplicates

In [166]:
# Create subset bank_dups to hold any duplicated rows in the bank dataframe
bank_dups = bank[bank.duplicated()]

# If bank_dups returns empty, there are no duplicates.
# If bank_dups returns values, those are the duplicates. 
bank_dups

,Row_Number,Customer_ID,Last_Name,Credit Score,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard?,IsActiveMember,Estimated Salary,ExitedFromBank?,Unnamed: 14,Unnamed: 15


There are no duplicates.

In [169]:
# Check data shape
bank.shape

(990, 16)

## 4. Data Wrangling

### 4.1 Droping unnecessary columns

In [173]:
bank.head()

,Row_Number,Customer_ID,Last_Name,Credit Score,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard?,IsActiveMember,Estimated Salary,ExitedFromBank?,Unnamed: 14,Unnamed: 15
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,True,True,101348.88,True,NaN,NaN
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,False,True,112542.58,False,NaN,NaN
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,True,False,113931.57,True,NaN,NaN
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,False,False,93826.63,False,NaN,NaN
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,True,True,79084.10,False,NaN,NaN


In [187]:
# Drop Unnecessary column
bank = bank.drop(columns=['Unnamed: 15'])

In [189]:
bank.head()

,Customer_ID,Last_Name,Credit Score,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard?,IsActiveMember,Estimated Salary,ExitedFromBank?
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,True,True,101348.88,True
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,False,True,112542.58,False
2,15619304,Onio,502,France,Female,42,8,159660.80,3,True,False,113931.57,True
3,15701354,Boni,699,France,Female,39,1,0.00,2,False,False,93826.63,False
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,True,True,79084.10,False


In [191]:
bank.shape

(990, 13)

## 5. Check frequencies in pertinent columns

### 5.1 Country

In [195]:
bank['Country'].value_counts()

Country
FR         244
France     236
Germany    234
Spain      135
ES         118
DE          23
Name: count, dtype: int64

In [197]:
# Update FR to France
bank['Country'] = bank['Country'].replace('FR', 'France')
# Update ES to Spain
bank['Country'] = bank['Country'].replace('ES', 'Spain')
# Update DE to Germany
bank['Country'] = bank['Country'].replace('DE', 'Germany')


In [199]:
# Confirm changes
bank['Country'].value_counts()

Country
France     480
Germany    257
Spain      253
Name: count, dtype: int64

### 5.2 Gender

In [202]:
bank['Gender'].value_counts()

Gender
Male      479
Female    443
M          49
F          19
Name: count, dtype: int64

In [204]:
# Update M to male
bank['Gender'] = bank['Gender'].replace('M', 'Male')
# Update F to female
bank['Gender'] = bank['Gender'].replace('F', 'Female')

In [206]:
# Confirm changes
bank['Gender'].value_counts()

Gender
Male      528
Female    462
Name: count, dtype: int64

### 5.3 Age

In [209]:
# Correct Age values <18 years
age_check = bank[bank['Age'] < 18]

In [211]:
age_check

,Customer_ID,Last_Name,Credit Score,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard?,IsActiveMember,Estimated Salary,ExitedFromBank?
624,15745307,Ch'iu,477,Spain,Female,2,2,129120.64,1,False,True,26475.79,False
629,15692463,Rahman,799,Spain,Female,2,3,142253.65,1,True,False,45042.56,False
632,15718673,Mirams,839,Spain,Female,2,10,75592.43,1,True,False,62674.42,False
635,15633648,Jideofor,696,Spain,Female,2,5,0.00,2,True,False,55022.43,False
636,15603323,Bell,660,Spain,Female,2,1,0.00,2,False,False,117834.91,False
673,15745621,Wertheim,640,Spain,Female,2,6,118879.35,2,True,True,19131.71,False
676,15607040,P'an,593,Spain,Female,2,4,88736.44,2,True,False,67020.03,False
678,15810978,Pugliesi,788,Spain,Female,2,1,0.00,2,True,True,41610.62,False
679,15668886,Blakey,684,Spain,Female,2,3,0.00,2,True,False,44255.65,False
702,15603436,Savage,594,Spain,Female,2,2,126615.94,2,False,True,123214.74,False


There are 11 records with ages less than 18 years. Need to be removed.

In [214]:
bank = bank.drop(age_check.index)

In [216]:
bank.describe()

,Customer_ID,Credit Score,Age,Tenure,Balance,NumOfProducts,Estimated Salary
count,9.790000e+02,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000
mean,1.569225e+07,648.372829,39.111338,5.084780,78174.555312,1.518897,99072.619871
std,7.223189e+04,97.922578,10.290507,2.929639,62847.279285,0.572370,57220.874890
min,1.556609e+07,376.000000,18.000000,0.000000,0.000000,1.000000,371.050000
25%,1.563019e+07,579.500000,32.000000,2.000000,0.000000,1.000000,50090.645000
50%,1.569275e+07,653.000000,37.000000,5.000000,99027.610000,1.000000,98820.390000
75%,1.575396e+07,721.000000,44.000000,8.000000,129500.955000,2.000000,146856.220000
max,1.581536e+07,850.000000,82.000000,10.000000,213146.200000,4.000000,199725.390000


Minimum age is 18 now.

## 6. Check shape, structure and basic descriptive statistics on after updates

In [220]:
bank.shape

(979, 13)

In [222]:
bank.head()

,Customer_ID,Last_Name,Credit Score,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard?,IsActiveMember,Estimated Salary,ExitedFromBank?
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,True,True,101348.88,True
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,False,True,112542.58,False
2,15619304,Onio,502,France,Female,42,8,159660.80,3,True,False,113931.57,True
3,15701354,Boni,699,France,Female,39,1,0.00,2,False,False,93826.63,False
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,True,True,79084.10,False


In [224]:
bank.describe()

,Customer_ID,Credit Score,Age,Tenure,Balance,NumOfProducts,Estimated Salary
count,9.790000e+02,979.000000,979.000000,979.000000,979.000000,979.000000,979.000000
mean,1.569225e+07,648.372829,39.111338,5.084780,78174.555312,1.518897,99072.619871
std,7.223189e+04,97.922578,10.290507,2.929639,62847.279285,0.572370,57220.874890
min,1.556609e+07,376.000000,18.000000,0.000000,0.000000,1.000000,371.050000
25%,1.563019e+07,579.500000,32.000000,2.000000,0.000000,1.000000,50090.645000
50%,1.569275e+07,653.000000,37.000000,5.000000,99027.610000,1.000000,98820.390000
75%,1.575396e+07,721.000000,44.000000,8.000000,129500.955000,2.000000,146856.220000
max,1.581536e+07,850.000000,82.000000,10.000000,213146.200000,4.000000,199725.390000


## 7. Export data

In [229]:
# Export data
bank.to_csv(os.path.join(path, '02. Data', 'Prepared Data', 'PigEBank_cleaned.csv'), index=False)